In [73]:
# %load psOnSale.py
import requests
from bs4 import BeautifulSoup
import json
import tablib
import time
import pandas as pd
    

In [74]:
#避免撈太快被擋IP
def sleeptime(hour,min,sec):
    return hour*3600 + min*60 + sec
#結合頁數與網址
def combineUrl (url,page):
    return str.format(url, page) 

In [75]:
#將資料組成table
def getSonyOnSale(url):
    page = 1
    formatedUrl = combineUrl(url,page)
    try:
        res = requests.get(formatedUrl)
        while(formatedUrl == res.url):
            soup = BeautifulSoup(res.text, 'html.parser')

            ItemList = soup.findAll('div',class_="grid-cell__body")

            for item in ItemList:
                yield {
                '遊戲名稱' : item.find('span').get('title'),
                '價錢' : item.h3.text,
                '連結' : "https://store.playstation.com" + item.find('a').get('href'),
                }
            page = page + 1
            formatedUrl = combineUrl(url,page)
            time.sleep(sleeptime(0,0,5))
            res = requests.get(formatedUrl)
    except:
        print("An exception occurred")

In [76]:
def start():
    url = input("請輸入特價網址:")
    url_fornt = url[:url.index('?')]
    url_end = url[url.index('?'):]
    url_fornt = url_fornt.split('/')
    url_fornt[-1] = "{0}"
    url_processed = '/'.join(url_fornt) + url_end
    items = getSonyOnSale(url_processed)
    s = json.dumps(list(items), indent = 4, ensure_ascii=False)
    rows = json.loads(s)
   # 将json中的key作为header, 也可以自定义header（列名）
    header=tuple([ i for i in rows[0].keys()])

    data = []
# 循环里面的字典，将value作为数据写入进去
    for row in rows:
        body = []
        for v in row.values():
            body.append(v)
        data.append(tuple(body))
    #可存成存成execel
    #data = tablib.Dataset(*data,headers=header)
    #open('psOnSale.xlsx', 'wb').write(data.export('xlsx'))   
    
    #網頁顯示用
    df = pd.DataFrame(data, columns =['遊戲名稱', '價錢', '連結']) 
    print(df.transpose())
    

In [77]:
start()

請輸入特價網址:https://store.playstation.com/zh-hant-tw/grid/STORE-MSF86012-PLUS_DIS_CONTENT/1?smcid=pdc%3Azh-hant-tw%3Acht-tw%3Aprimary%2520nav%3Amsg-games%3A
                                                      0  \
遊戲名稱                          眾神之鬪 (繁體中文, 日文, 英文, 簡體中文)   
價錢                                               NT$630   
連結    https://store.playstation.com/zh-hant-tw/produ...   

                                                      1  \
遊戲名稱                                       人體解剖學VR (英文)   
價錢                                               NT$690   
連結    https://store.playstation.com/zh-hant-tw/produ...   

                                                      2  \
遊戲名稱                               土撥鼠日：像父親一樣兒子 (日英韓文版)   
價錢                                               NT$451   
連結    https://store.playstation.com/zh-hant-tw/produ...   

                                                      3  \
遊戲名稱                         Genesis 奈莫勒斯-毀滅戰警禮包 (中英文版)   
價錢                